In [ ]:
import time
import opensg.utils as utils
from opensg.mesh.segment import SolidSegmentMesh
from opensg.core.solid import compute_stiffness
import opensg.core.stress_recov as stress_recov
tic = time.time()

segid=8 # user defined
# Read 3D yaml
filename='../data/Solid_3DSG/bar_urc_npl_1_ar_5-segment_'
mesh_yaml=filename+ str(segid) +'.yaml' 
segment_mesh = SolidSegmentMesh(mesh_yaml)

# Extract material parameters and mesh data
material_parameters, density = segment_mesh.material_database
meshdata = segment_mesh.meshdata
l_submesh, r_submesh = segment_mesh.left_submesh, segment_mesh.right_submesh

# Compute stiffness and mass matrices 
timo = compute_stiffness(material_parameters, meshdata, 
                         l_submesh, r_submesh, Taper=True)
taper_mass=utils.solid.get_mass_solid(meshdata,density,Taper=True)

print('Tapered Segment Origin:', meshdata["origin"])
print('\n Taper Mass \n',taper_mass)
print('\n Taper Stiffness \n', timo[0])
print('\n Time Taken',str(time.time()-tic))

In [ ]:
#Dehomogenization

beam_out=utils.beamdyn_trans.beam_reaction(filename)
# Local strain and local displacement 
strain_m,u_loc=stress_recov.local_strain(timo,beam_out,segid,meshdata) 

# Local stress (quadrature points) and local stress(elemental nodes)
stress_m_quad, coord_quad, stress_m_node, coord_node=stress_recov.stress_eval(material_parameters, meshdata, strain_m)

In [ ]:
# Eigenvalue
eigen= stress_recov.eigen_solve(material_parameters,
                                    meshdata,
                                    l_submesh,
                                    r_submesh,
                                    strain_m,
                                    u_loc)    
print('Computed Eigenvalue:', eigen)
print('\n Time Taken:',str(time.time()-tic))

In [ ]:
import dolfinx
stress_m.name = 'Stress' # Set a name for Paraview
#strain_m.name='Strain"
#u_local.name='Displacement"
filename='solid_segment'
with dolfinx.io.XDMFFile(meshdata['mesh'].comm, filename+'xmdf', "a") as xdmf:  
    xdmf.write_function(stress_m, 0.0)
   # xdmf.write_function(strain_m, 0.0)
   # xdmf.write_function(u_loc, 0.0)

In [ ]:
# Visualize plot
import pyvista
pyvista.start_xvfb()
u_topology, u_cell_types, u_geometry=dolfinx.plot.vtk_mesh(meshdata['mesh'],meshdata['mesh'].topology.dim)
grid = pyvista.UnstructuredGrid(u_topology, u_cell_types, u_geometry)
grid.cell_data["Marker"] = meshdata['subdomains'].values[:]
grid.set_active_scalars("Marker")
u_plotter = pyvista.Plotter()
u_plotter.add_mesh(grid)
u_plotter.view_yz() # x is beam axis
u_plotter.show()